Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers including convert model, generate input and performance test.

# Prerequisites

Pull dockers from Azure. It should take several minutes.

In [1]:
!sh build.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: Get https://ziylregistry.azurecr.io/v2/: unauthorized: authentication required
Using default tag: latest
latest: Pulling from onnx-converter
Digest: sha256:59366705ce552225e0e18e17dbc353d81414809c4a1d00c9ad5fc04674a49537
Status: Image is up to date for ziylregistry.azurecr.io/onnx-converter:latest
Using default tag: latest
latest: Pulling from perf-test
Digest: sha256:9b156a1dfd1a6c043cdd61980c2946ea9e236302660372b8a3e181e139340310
Status: Image is up to date for ziylregistry.azurecr.io/perf-test:latest


Install the onnxpipeline SDK

In [2]:
import onnxpipeline

pipeline = onnxpipeline.Pipeline()

# onnx
#pipeline = onnxpipeline.Pipeline('onnx')

# tensorflow
#pipeline = onnxpipeline.Pipeline('mnist/model')

# pytorch
#pipeline = onnxpipeline.Pipeline('pytorch')

# cntk 
#pipeline = onnxpipeline.Pipeline('cntk')

# keras
#pipeline = onnxpipeline.Pipeline('KerasToONNX')

# sklearn
#pipeline = onnxpipeline.Pipeline('sklearn')

# caffe
#pipeline = onnxpipeline.Pipeline('caffe')

# current directory
#pipeline = onnxpipeline.Pipeline()

# test mxnet fail
#pipeline = onnxpipeline.Pipeline('mxnet')


## Run parameters

(1) local_directory: string

    Required. The path of local directory where would be mounted to the docker. All operations will be executed from this path.

(2) mount_path: string

    Optional. The path where the local_directory will be mounted in the docker. Default is "/mnt/model".

(3) print_logs: boolean

    Optional. Whether print the logs from the docker. Default is True.
    
(4) convert_directory: string

    Optional. The directory path for converting model. Default is test/.    

(5) convert_name: string

    Optional. The model name for converting model. Default is model.onnx.
    


# Config information for ONNX pipeline

In [3]:
pipeline.config()

-----------config----------------
           Container information: <docker.client.DockerClient object at 0x7fedb7b5cbd0>
 Local directory path for volume: /home/chuche/notebook
Volume directory path in dockers: /mnt/model
                     Result path: result
        Converted directory path: test
        Converted model filename: model.onnx
            Converted model path: test/model.onnx
        Print logs in the docker: True


# Convert model to ONNX

This image is used to convert model from major model frameworks to onnx. Supported frameworks are - caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.


You can run the docker image with customized parameters.

In [4]:

model = pipeline.convert_model(model='onnx/perf-test/ImageQnA.onnx', model_type='onnx')

# test tensorflow
#model = pipeline.convert_model(model_type='tensorflow')

# test pytorch
#model = pipeline.convert_model(model_type='pytorch', model='saved_model.pb', model_input_shapes='(1,3,224,224)')

# test cntk
#model = pipeline.convert_model(model_type='cntk', model='ResNet50_ImageNet_Caffe.model')

# test keras
#model = pipeline.convert_model(model_type='keras', model='keras_Average_ImageNet.keras')

# test sklearn
#model = pipeline.convert_model(model_type='scikit-learn', model='sklearn_svc.joblib', initial_types=("float_input", "FloatTensorType([1,4])"))

# test caffe
#model = pipeline.convert_model(model_type='caffe', model='bvlc_alexnet.caffemodel')

# test mxnet
#model = pipeline.convert_model(model_type='mxnet', model='resnet.json', model_params='resnet.params', model_input_shapes='(1,3,224,224)')



-------------

Model Conversion



Input model is already ONNX model. Skipping conversion.



-------------

MODEL INPUT GENERATION(if needed)



Input.pb already exists. Skipping dummy input generation. 



-------------

MODEL CORRECTNESS VERIFICATION





Check the ONNX model for validity 

The ONNX model is valid.



Check ONNX model for correctness. 

The original model is already onnx. Skipping correctness test. 



-------------

MODEL CONVERSION SUMMARY (.json file generated at /mnt/model/test/output.json )



{'conversion_status': 'SUCCESS',

 'correctness_verified': 'SKIPPED',

 'error_message': '',

 'input_folder': '/mnt/model/test/test_data_set_0',

 'output_onnx_path': '/mnt/model/test/model.onnx'}



## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.

(2) output_onnx_path: string

    Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

    Required. The name of original model framework. Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs_names: string

    (tensorflow) Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs_names: string

    (tensorflow) Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string 

    (mxnet) Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple 

    (pytorch, mxnet) Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: int

    Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to 7.
    
(9) caffe_model_prototxt: string

    (caffe) Optional. The filename of deploy prototxt for the caffe madel. 

(10) initial_types: tuple (string, string)

    (scikit-learn) Optional. A tuple consist two strings. The first is data type and the second is the size of tensor type e.g., ('float_input', 'FloatTensorType([1,4])')

(11) input_json: string

    Optional. Use JSON file as input parameters.

(12) convert_json: boolean
    
    Optional. Convert all parameters into JSON file for input parameters.
    
(13) model_inputs_names: string

    (tensorflow) Optional.
    
(14) model_outputs_names: string

    (tensorflow) Optional.    


# Performance test tool

You can run perf_test using command python perf_test.py [Your model path] [Output path on the docker]. You can use the same arguments as for onnxruntime_pert_test tool, e.g. -m for mode, -e to specify execution provider etc. By default it will try all providers available.

In [5]:
result = pipeline.perf_test(model=model)
#result = pipeline.perf_test()   # is ok, too

Setting thread pool size to 1

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-ten

Total time cost:0.0186241

Total iterations:20

Average time cost:0.931206 ms

Setting thread pool size to 4

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_uti

Total time cost:0.0187147

Total iterations:20

Average time cost:0.935735 ms

Setting thread pool size to 5

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_uti

Total time cost:0.0186088

Total iterations:20

Average time cost:0.930442 ms

Setting thread pool size to 1

Total time cost:0.0201317

Total iterations:20

Average time cost:1.00658 ms

Setting thread pool size to 4

Total time cost:0.0208313

Total iterations:20

Average time cost:1.04157 ms

Setting thread pool size to 5

Total time cost:0.020207

Total iterations:20

Average time cost:1.01035 ms

2019-07-02 19:00:47.666888528 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-02 19:00:47.666938681 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1_

Setting thread pool size to 1

Total time cost:0.018548

Total iterations:20

Average time cost:0.927401 ms

2019-07-02 19:00:50.305263883 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-02 19:00:50.305310216 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1__64

2019-07-02 19:00:50.305334538 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/WordC

...

Setting thread pool size to 2

Total time cost:0.0186385

Total iterations:20

Average time cost:0.931925 ms

2019-07-02 19:00:55.573391297 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-02 19:00:55.573439243 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1__64

2019-07-02 19:00:55.573462783 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Word

Setting thread pool size to 3

Total time cost:0.0185134

Total iterations:20

Average time cost:0.925668 ms

Setting thread pool size to 0

Total time cost:0.019929

Total iterations:20

Average time cost:0.99645 ms

Setting thread pool size to 0

Total time cost:0.0186447

Total iterations:20

Average time cost:0.932235 ms

Setting thread pool size to 0

Total time cost:0.0156862

Total iterations:20

Average time cost:0.784312 ms

Setting thread pool size to 0

Total time cost:0.0153811

Total iterations:20

Average time cost:0.769054 ms

Setting thread pool size to 0

Total time cost:0.0199485

Total iterations:20

Average time cost:0.997426 ms

Setting thread pool size to 0

Total time cost:0.0251004

Total iterations:20

Average time cost:1.25502 ms

Setting thread pool size to 0

Total time cost:0.0292739

Total iterations:20

Average time cost:1.4637 ms

Setting thread pool size to 0

Total time cost:0.0200479

Total iterations:20

Average time cost:1.0024 ms

Setting thread po

/perf_test/bin/RelWithDebInfo/mkldnn_mklml/onnxruntime_perf_test -e mkldnn -x 3 -m times -r 20 /mnt/model/test/model.onnx /mnt/model/result/dda02f90-d793-4915-930a-8bbf5b85d370Setting thread pool size to 2

Total time cost:0.0276481

Total iterations:20

Average time cost:1.3824 ms

Setting thread pool size to 1

Total time cost:0.0257102

Total iterations:20

Average time cost:1.28551 ms

Setting thread pool size to 4

Total time cost:0.0199408

Total iterations:20

Average time cost:0.997039 ms

Setting thread pool size to 5

Total time cost:0.0191601

Total iterations:20

Average time cost:0.958006 ms

Setting thread pool size to 1

Total time cost:6.13481

Total iterations:20

Average time cost:306.74 ms

Setting thread pool size to 4

Total time cost:5.58809

Total iterations:20

Average time cost:279.404 ms

Setting thread pool size to 2

Total time cost:6.18402

Total iterations:20

Average time cost:309.201 ms

Setting thread pool size to 0

Found unsupport node: 0

Found unsup

Total time cost:0.0185871

Total iterations:20

Average time cost:0.929353 ms

Setting thread pool size to 0

Total time cost:0.0197451

Total iterations:20

Average time cost:0.987257 ms

2019-07-02 19:01:54.278642231 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-02 19:01:54.278691921 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1__64

2019-07-02 19:01:54.278715992 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_no

Setting thread pool size to 0

Total time cost:0.0183938

Total iterations:20

Average time cost:0.919688 ms

Setting thread pool size to 0

Total time cost:0.0155826

Total iterations:20

Average time cost:0.779131 ms

Setting thread pool size to 0

Total time cost:0.039948

Total iterations:20

Average time cost:1.9974 ms

Setting thread pool size to 0

Total time cost:0.0203433

Total iterations:20

Average time cost:1.01716 ms

Setting thread pool size to 0

Total time cost:14.7199

Total iterations:20

Average time cost:735.997 ms

Setting thread pool size to 0

Total time cost:0.0578132

Total iterations:20

Average time cost:2.89066 ms

Setting thread pool size to 0

Total time cost:5.95919

Total iterations:20

Average time cost:297.96 ms

Setting thread pool size to 0

Total time cost:0.0219379

Total iterations:20

Average time cost:1.0969 ms

Setting thread pool size to 0

Total time cost:0.0179228

Total iterations:20

Average time cost:0.896138 ms

Setting thread pool size

# Run parameters

(1) model: string

    Required. The path of the model that wants to be performed.
    
(2) result: string

    Optional. The path of the result.
    
(3) config: string (choices=["Debug", "MinSizeRel", "Release", "RelWithDebInfo"])

    Optional. Configuration to run. Default is "RelWithDebInfo".
    
(4) mode: string (choices=["duration", "times"])

    Optional. Specifies the test mode. Value could be 'duration' or 'times'. Default is "times".

(5) execution_provider: string (choices=["cpu", "cuda", "mkldnn"])

    Optional. help="Specifies the provider 'cpu','cuda','mkldnn'. Default is ''.
    
(6) repeated_times: integer

    Optional. Specifies the repeated times if running in 'times' test mode. Default:20.
    
(7) duration_times: integer

    Optional. Specifies the seconds to run for 'duration' mode. Default:10.
    
(8) parallel: boolean

    Optional. Use parallel executor, default (without -x): sequential executor.
    
(9) threadpool_size: integer

    Optional. Threadpool size if parallel executor (--parallel) is enabled. Default is the number of cores.
    
(10) num_threads: integer

    Optional. OMP_NUM_THREADS value.
    
(11) top_n: integer

    Optional. Show percentiles for top n runs. Default:5.
    
(12) runtime: boolean

    Optional. Use this boolean flag to enable GPU if you have one.
    
(13) input_json: string

    Optional. Use JSON file as input parameters.
    
(14) convert_json: boolean

    Optional. Convert all parameters into JSON file for input parameters.
   

In [6]:
pipeline.print_performance(result)
#pipeline.print_result() # is ok, too

mkldnn_openmp_6_threads 0.00059004 s

mkldnn_openmp_6_threads_OMP_WAIT_POLICY_active 0.000592109 s

cpu_openmp_5_threads 0.000656048 s

mkldnn_openmp_5_threads_OMP_WAIT_POLICY_active 0.000662145 s

cpu_openmp_5_threads_OMP_WAIT_POLICY_active 0.000663101 s

cpu_openmp_6_threads 0.000791471 s

cpu_openmp_6_threads_OMP_WAIT_POLICY_active 0.000796548 s

mkldnn_openmp_5_threads 0.000803457 s

cpu 0.000895772 s

cpu_openmp 0.000908881 s

cuda 0.000917489 s

cuda_2_threads 0.000925966 s

tensorrt_4_threads 0.000929724 s

cuda_1_threads 0.00093062 s

cuda_3_threads 0.000930718 s

tensorrt 0.000931352 s

tensorrt_1_threads 0.000932568 s

cuda_4_threads 0.000934084 s

mkldnn_openmp_4_threads 0.000936962 s

tensorrt_5_threads 0.00093819 s

mkldnn_openmp_4_threads_OMP_WAIT_POLICY_active 0.000941316 s

cpu_openmp_4_threads 0.000969576 s

cpu_openmp_4_threads_OMP_WAIT_POLICY_active 0.000970837 s

ngraph_4_threads 0.000996577 s

ngraph 0.000998741 s

mkldnn_openmp_1_threads_OMP_WAIT_POLICY_active 0.0

# Result Visualization

After performance test, there would be a directory for results. 

This libray use Pandas.read_json to visualize JSON file. (orient is changeable.)

"latency.json" contains the raw data of results ordered by the average time. 

Use .latency to obtain the original latency JSON; Use .profiling to obtain original top 5 profiling JSON.

In [7]:
r = pipeline.get_result(result)
#r.latency
#r.profiling

# Print latency.json

Provide parameters for top 5 performace. Use the parameter "top" to visualize more results.

In [8]:
r.prints()

,avg,code_snippet,command,cpu_usage,gpu_usage,memory_util,name,p90,p95
0,0.000590,{u'environment_variables': {u'LD_LIBRARY_PATH'...,/perf_test/bin/RelWithDebInfo/mkldnn_openmp/on...,0.504306,0,0.041267,mkldnn_openmp_6_threads,0.000695,0.000781
1,0.000592,{u'environment_variables': {u'OMP_WAIT_POLICY'...,/perf_test/bin/RelWithDebInfo/mkldnn_openmp/on...,0.576842,0,0.041242,mkldnn_openmp_6_threads_OMP_WAIT_POLICY_active,0.000707,0.000779
2,0.000656,{u'environment_variables': {u'LD_LIBRARY_PATH'...,/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxr...,0.540563,0,0.041298,cpu_openmp_5_threads,0.000674,0.000777
3,0.000662,{u'environment_variables': {u'OMP_WAIT_POLICY'...,/perf_test/bin/RelWithDebInfo/mkldnn_openmp/on...,0.524025,0,0.041217,mkldnn_openmp_5_threads_OMP_WAIT_POLICY_active,0.000716,0.000777
4,0.000663,{u'environment_variables': {u'OMP_WAIT_POLICY'...,/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxr...,0.552111,0,0.041251,cpu_openmp_5_threads_OMP_WAIT_POLICY_active,0.000770,0.000777


# Print profiling.json

Only provide profiling JSON for top 5 performace by giving certain index of the result. The file name is profile_[name].json

In [9]:
r.print_profiling(4)

,cat,dur,name,op_name,ph,pid,provider,tid,ts
0,Session,51456,model_loading_uri,NaN,X,1800,NaN,1800,1
1,Session,52803,session_initialization,NaN,X,1800,NaN,1800,51463
2,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1800,NaN,1800,104427
3,Node,522,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1800,CPUExecutionProvider,1800,104430
4,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1800,NaN,1800,104956
5,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Cast,X,1800,NaN,1800,104959
6,Node,2,agi.lib.tf.model_repository.spacev_co_train_no...,Cast,X,1800,CPUExecutionProvider,1800,104961
7,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Cast,X,1800,NaN,1800,104965
8,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Reshape,X,1800,NaN,1800,104967
9,Node,1,agi.lib.tf.model_repository.spacev_co_train_no...,Reshape,X,1800,CPUExecutionProvider,1800,104968


# Get environment_variables in code_snippet

Given the index from latency.json, output the environment variables.

In [10]:
r.print_environment(3)

,0
LD_LIBRARY_PATH,/perf_test/bin/RelWithDebInfo/mkldnn_openmp
OMP_WAIT_POLICY,active


# Get codes

Given the index from latency.json, output the codes.

In [11]:
print(r.get_code(3))


import onnxruntime as ort
so = rt.SessionOptions()
so.set_graph_optimization_level(2)
so.enable_sequential_execution = True
so.session_thread_pool_size(0)
session = rt.Session("/mnt/model/test/model.onnx", so)



# netron

In [ ]:
# only workable for notebook in the local server 
import netron
netron.start(model) # 'model.onnx'
from IPython.display import IFrame
IFrame('http://localhost:8080', width=700, height=350)